<a href="https://colab.research.google.com/github/aliash98/Search2Vec/blob/main/SkipGram_Validation_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialization

In [ ]:
!pip install pandas
!pip install numpy

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
import json
import warnings
import gc 
from tqdm import tqdm, trange
from scipy import spatial
import math

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = pd.read_csv(r'./gdrive/MyDrive/Colab Notebooks/learned_weights_productbased.tsv', sep="\t")   
weights = df.values

# *** Note : this weights matrix doesn't contain the 0th redundant index

print(weights.shape)
print(weights[:5, :5])

(7266, 300)
[[-0.13176316  0.06450842  0.04117316  0.09357771 -0.10613987]
 [ 0.13581131  0.06118518  0.06497271  0.09971684  0.11181994]
 [-0.09656929  0.06580082 -0.1086641   0.08446457  0.02570293]
 [ 0.09812609 -0.12900965 -0.13850017  0.06715225 -0.10543239]
 [-0.15182501  0.08127624  0.07283206  0.12366455 -0.11300164]]


In [ ]:
vocab = json.load( open( r'./gdrive/MyDrive/Colab Notebooks/vocab_productbased.json' ) )
print(type(vocab))
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab[1])

<class 'dict'>
موبایل


## Finding the least cosine distance for a point

In [ ]:
def least_cosine_distance(index, k):
  vocab_size, dim = weights.shape
  vector = weights[index, :]
  # print(vector.shape)
  # lets find the k nearest point to this
  distances = []
  indexes = []
  min_distances = []
  for i in range(0, vocab_size):
    distances.append(spatial.distance.cosine(vector, weights[i, :]))
  counter = 0
  while counter < k:
    temp_index = np.argmin(distances)
    if temp_index == index:
      distances[temp_index] = 1
      continue
    if not inverse_vocab[temp_index].endswith('-prod'):
      distances[temp_index] = 1
      continue
    indexes.append(temp_index + 1)
    min_distances.append(distances[temp_index])
    distances[temp_index] = 1
    counter = counter + 1
  return indexes, min_distances 

    

In [ ]:
vocab['هارد اکسترنال']

42

In [ ]:
my_point = 42
indexes, min_distances = least_cosine_distance(my_point - 1, 10)
print(inverse_vocab[my_point])
print(min_distances)
print(indexes)
for index in indexes:
  print(inverse_vocab[index])
# a = weights[33, :]
# b = weights[714, :]
# print(1 - np.dot(a, b)/(np.linalg.norm(a) * np.linalg.norm(b)))

هارد اکسترنال
[0.06177504337191009, 0.07163379492400113, 0.07357228260164206, 0.07426750791217762, 0.07558182972033023, 0.07700407940401566, 0.07701380002998082, 0.0789112422859547, 0.08323561698513049, 0.08544488216035717]
[5856, 1777, 1124, 3612, 1758, 3157, 1955, 1588, 5946, 6209]
mmngy-prod
mmngf-prod
mmngk-prod
mmngz-prod
mmnvf-prod
mmnga-prod
mmocp-prod
mmnvd-prod
mmika-prod
mmini-prod


## Getting the validation set

In [ ]:
validation_query = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/most_freq_queries_validation.csv')
validation_clicks = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/most_freq_clicks_validation.csv')
validation_products = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/most_freq_products_validation.csv')

In [ ]:
# Rename
validation_clicks.rename(columns={'search_log_id':'_id'}, inplace=True)

In [ ]:
print(validation_query.info())
print(validation_clicks.info())
print(validation_products.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243638 entries, 0 to 243637
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   _id        243638 non-null  object
 1   raw_query  243638 non-null  object
 2   datetime   243638 non-null  object
dtypes: object(3)
memory usage: 5.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395070 entries, 0 to 395069
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   datetime         395070 non-null  object
 1   _id              395070 non-null  object
 2   rank             395070 non-null  int64 
 3   base_product_id  395070 non-null  object
dtypes: int64(1), object(3)
memory usage: 12.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45544 entries, 0 to 45543
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   _id 

## 10 nearaset category/product vocabulary

In [ ]:
nearest_cat_vocab = {}
counter = 0
for word in vocab.keys():
  if not word.endswith('-prod'):
    counter += 1
    print(counter)
    indexes, min_distances = least_cosine_distance(int(vocab[word]) - 1, 10)
    temp_list = []
    for index in indexes:
      temp_list.append(inverse_vocab[index])
    nearest_cat_vocab[word] = temp_list
  
print(nearest_cat_vocab)

## Constructing the list of logs

In [ ]:
logs = []
for i in range(1, 11):
  logs.append(1 / math.log(i + 1, 2))
print(logs)

[1.0, 0.6309297535714574, 0.5, 0.43067655807339306, 0.38685280723454163, 0.3562071871080222, 0.3333333333333333, 0.3154648767857287, 0.30102999566398114, 0.2890648263178878]


## Merging all datasets

In [ ]:
# *** Note: next lines are for categorical mode

# validation_products.rename(columns={'_id':'base_product_id'}, inplace=True)
# merged_click_product = pd.merge(validation_clicks, validation_products, how='inner', on=["base_product_id"])
# print(merged_click_product.info())
# merged_click_product = merged_click_product.drop_duplicates(subset=['_id'])

merged_click_product = validation_clicks.drop_duplicates(subset=['_id'])
total_df = pd.merge(validation_query, merged_click_product, how='inner', on=["_id"])
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243638 entries, 0 to 243637
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   _id              243638 non-null  object
 1   raw_query        243638 non-null  object
 2   datetime_x       243638 non-null  object
 3   datetime_y       243638 non-null  object
 4   rank             243638 non-null  int64 
 5   base_product_id  243638 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.0+ MB


## Computing the nDCG

In [ ]:
N, M = total_df.shape
print(N)

first_hit = 0
first_hit_default_method = 0
ndcg = 0
ndcg_default_method = 0

for i in range(N):
  temp_query = total_df.iloc[i, 1]
  # temp_category = total_df.iloc[i, 6] + 'cat'
  temp_product = total_df.iloc[i, 5] + '-prod'

  for j in range(10):
    if nearest_cat_vocab[temp_query][j] == temp_product:
      if j == 0:
        first_hit = first_hit + 1
      ndcg = ndcg + logs[j]
      break
  temp_rank = int(total_df.iloc[i, 4])
  if temp_rank < 10:
    ndcg_default_method = ndcg_default_method + logs[temp_rank]
    if temp_rank == 0:
        first_hit_default_method = first_hit_default_method + 1
        # print('hi')
    # print(i)
print(ndcg)





243638
43845.48686319846


In [ ]:
print('Observed model nDCG, according to the first clicks of each query, is ' + "{:.2f}".format(ndcg/N * 100) + '%')
print('Observed default nDCG, according to the first clicks of each query, is ' + "{:.2f}".format(ndcg_default_method/N * 100) + '%')

Observed model nDCG, according to the first clicks of each query, is 18.00%
Observed default nDCG, according to the first clicks of each query, is 27.53%


In [ ]:
print('Observed first hit, according to the model, is ' + "{:.2f}".format(first_hit/N * 100) + '%')
print('Observed first hit, according to the default model, is ' + "{:.2f}".format(first_hit_default_method/N * 100) + '%')

Observed first hit, according to the model, is 6.81%
Observed first hit, according to the default model, is 11.54%


In [ ]:
x, y = validation_products.shape
print(x)
plus = 0
for k in range(x-1):
  if (validation_products.iloc[k, 0] + '-prod') not in vocab.keys():
    plus += 1
print(plus)

45544
36162


In [ ]:
# query_cat_occur = merged_q_c.groupby(merged_q_c.columns.tolist(),as_index=False).size()


# merged_q_c['occur'] = merged_q_c.groupby('raw_query')['category_name'].transform('size')
# print(query_cat_occur.head(5))

    raw_query            category_name  size
0  12 pro max                شارژر اپل     2
1  12 pro max    محافظ صفحه گوشی آیفون     2
2  12 pro max   کیف و کاور گوشی و تبلت     9
3  12 pro max          گوشی اپل، آیفون   468
4         206  سایر لوازم تزئینی خودرو     1


In [ ]:
# print(query_cat_occur[query_cat_occur['raw_query'] == 'ماشین'])

     raw_query                  category_name  size
1942     ماشین                    اسکناس شمار     1
1943     ماشین         تجهیزات آشپزخانه صنعتی     7
1944     ماشین                دستگاه برش کاغذ     1
1945     ماشین                   دستگاه ماینر     1
1946     ماشین  دستگاه‌های بسته‌بندی و شیرینگ     2
1947     ماشین        سایر برندهای اسپرسو ساز     1
1948     ماشین               فایل‌های دانلودی     4
1949     ماشین               قطعات الکترونیکی     2
1950     ماشین          لوازم جانبی ابزارآلات     3
1951     ماشین        ماشین اصلاح و ریش‌ تراش   313
1952     ماشین                    ماشین شارژی     2
1953     ماشین                ماشین ظرف‌‌شویی     1
1954     ماشین   ماشین، قطار، کشتی اسباب بازی   729
